# Hyperparameter Tuning using HyperDrive

In [2]:
import os
import sys
import json
import azureml
import logging
import requests
import pandas as pd
import numpy as np
from io import BytesIO
import joblib

from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from azureml.core import Environment

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.estimator import Estimator
from azureml.core import ScriptRunConfig
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

from azureml.core.dataset import Dataset
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling, BayesianParameterSampling
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, quniform, choice

from azureml.core.runconfig import RunConfiguration
from azureml.core.runconfig import EnvironmentDefinition
from azureml.core.runconfig import CondaDependencies
from azureml.core.model import InferenceConfig, Model

from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model, InferenceConfig

from azureml.train.automl import constants

from train import *

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', None)

# Check system and core SDK version number
print("System version: {}".format(sys.version))
print("SDK version:", azureml.core.VERSION)

System version: 3.6.13 |Anaconda, Inc.| (default, Feb 23 2021, 12:58:59) 
[GCC Clang 10.0.0 ]
SDK version: 1.23.0


In [3]:
!az login

The default web browser has been opened at https://login.microsoftonline.com/common/oauth2/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.
You have logged in. Now let us find all the subscriptions to which you have access...
[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "660b3398-b80e-49d2-bc5b-ac1dc93b5254",
    "id": "2c48c51c-bd47-40d4-abbe-fb8eabd19c8c",
    "isDefault": true,
    "managedByTenants": [],
    "name": "Udacity CloudLabs Sub - 03",
    "state": "Enabled",
    "tenantId": "660b3398-b80e-49d2-bc5b-ac1dc93b5254",
    "user": {
      "name": "odl_user_142985@udacitylabs.onmicrosoft.com",
      "type": "user"
    }
  }
]


In [4]:
interactive_auth = InteractiveLoginAuthentication(tenant_id="660b3398-b80e-49d2-bc5b-ac1dc93b5254")
ws = Workspace.get(subscription_id="2c48c51c-bd47-40d4-abbe-fb8eabd19c8c",
                   resource_group="aml-quickstarts-142985",
                   name="quick-starts-ws-142985",
                   auth=interactive_auth)

experiment_name = 'online_news_project'
experiment=Experiment(ws, experiment_name)
experiment
# ws = Workspace.from_config()
# print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

Name,Workspace,Report Page,Docs Page
online_news_project,quick-starts-ws-142985,Link to Azure Machine Learning studio,Link to Documentation


In [5]:
dic_data = {'Workspace name': ws.name,
            'Azure region': ws.location,
            'Subscription id': ws.subscription_id,
            'Resource group': ws.resource_group,
            'Experiment Name': experiment.name}

az_data = pd.DataFrame.from_dict(data = dic_data, orient='index')
az_data.rename(columns={0:''}, inplace = True)
az_data

,
Workspace name,quick-starts-ws-142985
Azure region,southcentralus
Subscription id,2c48c51c-bd47-40d4-abbe-fb8eabd19c8c
Resource group,aml-quickstarts-142985
Experiment Name,online_news_project


## Create or Attach an AmlCompute cluster

In [6]:
# Define CPU cluster name
compute_target_name = "cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=compute_target_name)
    print("Found existing cpu-cluster. Use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_DS12_V2",
                                                           min_nodes=1, 
                                                           max_nodes=4) 
    compute_target = ComputeTarget.create(ws, compute_target_name, compute_config)

compute_target.wait_for_completion(show_output=True)
print(compute_target.get_status().serialize())

Found existing cpu-cluster. Use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 1, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 1, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-04-18T01:00:23.900000+00:00', 'errors': None, 'creationTime': '2021-04-18T00:58:49.540122+00:00', 'modifiedTime': '2021-04-18T00:59:05.307569+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 1, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_DS12_V2'}


## Dataset

The dataset used in this project is a dataset made available on UCI Machine Learning Repository called [Online News Popularity Data Set](https://archive.ics.uci.edu/ml/datasets/Online+News+Popularity#).

The dataset summarizes heterogeneous set of features about the articles published by Mashable between 2013 and 2015.

- Number of Instances: 39797
- Number of Attributes: 61 
    - 58 predictive attributes 
    - 2 non-predictive (`url` and `timedelta`) 
    - 1 target column
    
We will also apply the `Boruta` step for feature selection before exporting the data to our defined Datastore

In [7]:
DATA_LOC = "https://raw.githubusercontent.com/franckess/AzureML_Capstone/main/data/OnlineNewsPopularity.csv"
BORUTA_LOC = "https://github.com/franckess/AzureML_Capstone/releases/download/1.1/boruta_model_final.pkl"

# Loading data
df = pd.read_csv(DATA_LOC)

# Removing space character in the feature names
df.columns=df.columns.str.replace(' ','')

# Drop URL column
df = df.drop(['url'], axis=1)

# Perform Data pre-processing
df = corr_drop_cols(df)
df = create_label(df)
df = scaling_num(df)
df = feature_selection(df, BORUTA_LOC)
    
# Split train data into train & test
X_train, X_test, y_train, y_test = split_train_test(df)

m, k = X_train.shape
print("{} x {} table of data:".format(m, k))
X_train.info()

31715 x 47 table of data:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 31715 entries, 38512 to 35050
Data columns (total 47 columns):
n_tokens_title                   31715 non-null float64
n_tokens_content                 31715 non-null float64
n_unique_tokens                  31715 non-null float64
num_hrefs                        31715 non-null float64
num_self_hrefs                   31715 non-null float64
num_imgs                         31715 non-null float64
num_videos                       31715 non-null float64
average_token_length             31715 non-null float64
num_keywords                     31715 non-null float64
data_channel_is_entertainment    31715 non-null int64
data_channel_is_bus              31715 non-null int64
data_channel_is_socmed           31715 non-null int64
data_channel_is_tech             31715 non-null int64
data_channel_is_world            31715 non-null int64
kw_min_min                       31715 non-null float64
kw_max_min                     

## Upload data to Azure Datatore

In [8]:
datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./data/OnlineNewsPopularity.csv'], 
                       target_path='data/', 
                       overwrite=True, 
                       show_progress=True)

Uploading an estimated of 1 files
Uploading ./data/OnlineNewsPopularity.csv
Uploaded ./data/OnlineNewsPopularity.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_55737bc15eb54d20a6a645a5ffec8218

In [9]:
print(
    "Datastore type: " + datastore.datastore_type,
    "Account name: " + datastore.account_name,
    "Container name: " + datastore.container_name,
    sep="\n",
)

Datastore type: AzureBlob
Account name: mlstrg142985
Container name: azureml-blobstore-4d8774c1-27e9-47c1-b92b-547a17c80a82


In [10]:
# Get data reference object for the data path
ds_data = datastore.path('data/')
print(ds_data)

$AZUREML_DATAREFERENCE_f88cf551a9d7444580c30286a23fa4d2


## HyperDrive Configuration

### Create an environment

Define a conda environment YAML file with your training script dependencies and create an Azure ML environment.

In [ ]:
%%writefile hyperdrive_dependencies.yml

name: project_environment
dependencies:
- python=3.6.2
- pip:
  - azureml-train-automl-runtime==1.24.0
  - Werkzeug==0.16.1
  - inference-schema
  - azureml-interpret==1.24.0
  - azureml-defaults==1.24.0
  - pingouin
  - lightgbm
  - joblib  
  - Boruta
- numpy>=1.16.0,<1.19.0
- pandas==0.25.1
- scikit-learn==0.22.1
- py-xgboost<=0.90
- fbprophet==0.5
- holidays==0.9.11
- psutil>=5.2.2,<6.0.0
channels:
- anaconda
- conda-forge

In [11]:
udacity_env = Environment.from_conda_specification(name = 'udacity-env', file_path = './hyperdrive_dependencies.yml')
udacity_env.register(ws)

# Specify an Ubuntu base image
udacity_env.docker.enabled = True
udacity_env.python.user_managed_dependencies = False
udacity_env.docker.base_image = 'mcr.microsoft.com/azureml/base:openmpi3.1.2-ubuntu18.04'

Build the image just to confirm it works appropriately or identify any errors prior to deployment

In [ ]:
# build = udacity_env.build(workspace=ws)
# build.wait_for_completion(show_output=True)

In [12]:
details = udacity_env.get_image_details(ws)
print(details.dockerfile)

FROM mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04:20210129.v1@sha256:19fe6a6202757242643beace8d3d690519f431deb21adc072989185425a22133
USER root
RUN mkdir -p $HOME/.cache
WORKDIR /
COPY azureml-environment-setup/99brokenproxy /etc/apt/apt.conf.d/
RUN if dpkg --compare-versions `conda --version | grep -oE '[^ ]+$'` lt 4.4.11; then conda install conda==4.4.11; fi
COPY azureml-environment-setup/mutated_conda_dependencies.yml azureml-environment-setup/mutated_conda_dependencies.yml
RUN ldconfig /usr/local/cuda/lib64/stubs && conda env create -p /azureml-envs/azureml_d32c6a42b04d5ead1baaef3bc63324e7 -f azureml-environment-setup/mutated_conda_dependencies.yml && rm -rf "$HOME/.cache/pip" && conda clean -aqy && CONDA_ROOT_DIR=$(conda info --root) && rm -rf "$CONDA_ROOT_DIR/pkgs" && find "$CONDA_ROOT_DIR" -type d -name __pycache__ -exec rm -rf {} + && ldconfig
# AzureML Conda environment name: azureml_d32c6a42b04d5ead1baaef3bc63324e7
ENV PATH /azureml-envs/azureml_d32c6a42b04d5ead1baaef

### Tune hyperparameters using `HyperDrive`

In the following block, I tune my set of hyperparameters for the `LightGBM` model. The ranges of parameters for the `LightGBM` used were chosen considering the parameters tuning guides for different scenarios provided here.

The code below does a parallel search of the hyperparameter space using a Bayesian sampling method which does not support termination policy. Therefore, `policy=None`.

> __Note that when using Bayesian sampling, the number of concurrent runs has an impact on the effectiveness of the tuning process. Typically, a smaller number of concurrent runs leads to better sampling convergence. That is because some runs start without fully benefiting from runs that are still running.__

In order to compare the performance of HyperDrive with the one of AutoML we chose as objective metric of `LightGBM` __Accuracy__ score.

In [13]:
# Create a SKLearn estimator for use with train.py
src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=compute_target,
                      environment=udacity_env)

# Specify hyperparameter space
param_sampling = BayesianParameterSampling(
    {
        "--num-leaves": quniform(8, 128, 1),
        "--min-data-in-leaf": quniform(20, 500, 10),
        "--learning-rate": choice(
            1e-4, 1e-3, 5e-3, 1e-2, 1.5e-2, 2e-2, 3e-2, 5e-2, 1e-1
        ),
        "--feature-fraction": uniform(0.1, 1),
        "--bagging-fraction": uniform(0.1, 1),
        "--bagging-freq": quniform(1, 30, 1),
        "--max-depth": quniform(5, 50, 5)
    }
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=param_sampling,
                                     policy=None ,
                                     primary_metric_name="Accuracy",
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=50,
                                     max_concurrent_runs=10)

In [14]:
# Submit hyperdrive run to the experiment 
hyperdrive_run = experiment.submit(config=hyperdrive_config, show_output=True)

AzureMLException: AzureMLException:
	Message: Failed to flush task queue within 300 seconds
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Failed to flush task queue within 300 seconds",
        "inner_error": {
            "code": "ResourceExhausted",
            "inner_error": {
                "code": "Timeout"
            }
        }
    }
}

## Run Details

In [ ]:
# Show run details with the Jupyter widget
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)
hyperdrive_run.get_metrics()

## Retrieve and Save Best Model

Here I retrieve and save the best model as well as display all the properties of the model.

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric() 
get_best_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()["runDefinition"]["arguments"]

print('Best Run ID: ', best_run.id, sep='\n')
print('\n Metrics: ', get_best_metrics)
print('\n Best Run Accuracy:', get_best_metrics['Accuracy'])
print('\n Best model hyperparameter values', parameter_values, sep='\n')

In [ ]:
best_run.get_file_names()

In [ ]:
best_run.download_file("outputs/lgb_model.pkl","output/hyperdrive_model.pkl")

## Model Deployment

### Register our best model

In [ ]:
model = best_run.register_model(model_name = 'best_model', model_path = './outputs/lgb_model.pkl')
print("Model successfully registered.")

In [ ]:
print('Name:', model.name)
print('Version:', model.version)

In [ ]:
Model.get_model_path(model_name = 'best_model', _workspace=ws)

### Prepare the `score.py` script

In [ ]:
with open('score.py') as f:
    print(f.read())

### Inference configuration

Create an inference config and deploy the model as a web service.

In [ ]:
script_file_name = './score.py'
inference_config = InferenceConfig(entry_script=script_file_name)

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 2, 
                                               memory_gb = 4, 
                                               tags = {'Company': "Mashable", 'Type': "Hyperdrive", "Version":"1"}, 
                                               description = 'sample service for Capstone Project Hyperdrive Classifier for Online News popularity')
aci_service_name = 'hyperdrive-final-deployment'
print(aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(f'\nservice state: {aci_service.state}\n')
print(f'scoring URI: \n{aci_service.scoring_uri}\n')
print(f'swagger URI: \n{aci_service.swagger_uri}\n')

### Testing our web service

Let's send a request to the web service we deployed to test it.

In [ ]:
test_data = pd.concat([X_test, y_test], axis=1)
test_data = test_data[10:15]
display(test_data)

In [ ]:
# remove label column
label = test_data.pop('label')

# convert test input data to dictionary form
input_data = json.dumps({'data': test_data.to_dict(orient='records')})

# print test input data
print(input_data)

In [ ]:
# Set the content type
headers = {'Content-Type': 'application/json'}

# Make the request and display the response
resp = requests.post(aci_service.scoring_uri, input_data, headers=headers)

print("Response Code : ", resp.status_code)
print("Predicted Value : ",resp.json())

In [ ]:
output = aci_service.run(test_data)
print(output)